In [1]:
!pip install --no-deps seqeval[gpu]

In [2]:
%who

Interactive namespace is empty.


In [3]:
import numpy as np
import pandas as pd

import spacy
from spacy.gold import biluo_tags_from_offsets
nlp = spacy.load("en_core_web_lg")

from tqdm import trange
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
!pip install pytorch-pretrained-bert
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

from seqeval.metrics import classification_report, accuracy_score, f1_score

In [4]:
# Adding '\n' to the default spacy tokenizer

prefixes = ('\\n', ) + nlp.Defaults.prefixes
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [5]:
# Personal Custom Tags Dictionary
entity_dict = {
    'Name': 'NAME', 
    'College Name': 'CLG',
    'Degree': 'DEG',
    'Graduation Year': 'GRADYEAR',
    'Years of Experience': 'YOE',
    'Companies worked at': 'COMPANY',
    'Designation': 'DESIG',
    'Skills': 'SKILLS',
    'Location': 'LOC',
    'Email Address': 'EMAIL'
}

In [6]:
# loading the dataset
df = pd.read_json('Entity Recognition in Resumes.json', lines=True)
df.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [7]:
# Checking for unique values present in 'extras' column
df['extras'].unique()

array([nan])

In [8]:
# Since, 'extras' column contains no information we can drop the column
df = df.drop(['extras'], axis=1)
df.head()

,content,annotation
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20..."


In [9]:
def mergeIntervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)

    return merged

In [10]:
# From 'annotation' column, we are extracting the starting index, ending index, entity label
# So that we can convert the content in BILOU format

def get_entities(df):
    
    entities = []
    
    for i in range(len(df)):
        entity = []
    
        for annot in df['annotation'][i]:
            try:
                ent = entity_dict[annot['label'][0]]
                start = annot['points'][0]['start']
                end = annot['points'][0]['end'] + 1
                entity.append((start, end, ent))
            except:
                pass
    
        entity = mergeIntervals(entity)
        entities.append(entity)
    
    return entities

In [11]:
# Adding a new column 'entities'
df['entities'] = get_entities(df)
df.head()

,content,annotation,entities
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...","[(0, 12, NAME), (13, 46, DESIG), (49, 58, COMP..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...","[(0, 14, NAME), (62, 68, LOC), (104, 148, EMAI..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...","[(0, 21, NAME), (22, 31, LOC), (65, 117, EMAIL..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...","[(0, 12, NAME), (13, 51, DESIG), (54, 60, COMP..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...","[(0, 13, NAME), (14, 22, DESIG), (24, 41, COMP..."


In [12]:
def get_train_data(df):
    tags = []
    sentences = []

    for i in range(len(df)):
        text = df['content'][i]
        entities = df['entities'][i]
    
        doc = nlp(text)
    
        tag = biluo_tags_from_offsets(doc, entities)
        tmp = pd.DataFrame([list(doc), tag]).T
        loc = []
        for i in range(len(tmp)):
            if tmp[0][i].text is '.' and tmp[1][i] is 'O':
                loc.append(i)
        loc.append(len(doc))
    
        last = 0
        data = []
        for pos in loc:
            data.append([list(doc)[last:pos], tag[last:pos]])
            last = pos
    
        for d in data:
            tag = ['O' if t is '-' else t for t in d[1]]
            if len(set(tag)) > 1:
                sentences.append(d[0])
                tags.append(tag)
    
    return sentences, tags

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:26: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:26: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-12-e145e49fad27>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if tmp[0][i].text is '.' and tmp[1][i] is 'O':
<ipython-input-12-e145e49fad27>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if tmp[0][i].text is '.' and tmp[1][i] is 'O':
<ipython-input-12-e145e49fad27>:26: SyntaxWarning: "is" with a literal. Did you mean "=="?
  tag = ['O' if t is '-' else t for t in d[1]]


In [13]:
sentences, tags = get_train_data(df)
len(sentences), len(tags)

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Afreen Jamadar
Active member of IIIT Committee in ..." with entities "[(0, 14, 'NAME'), (62, 68, 'LOC'), (104, 148, 'EMA...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Akhil Yadav Polemaina
Hyderabad, Telangana - Email..." with entities "[(0, 21, 'NAME'), (22, 31, 'LOC'), (65, 117, 'EMAI...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Alok

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Imgeeyaul Ansari
java developer

Pune, Maharashtra..." with entities "[(0, 16, 'NAME'), (17, 31, 'DESIG'), (33, 37, 'LOC...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Jay Madhavi
Navi Mumbai, Maharashtra - Email me on..." with entities "[(0, 11, 'NAME'), (12, 23, 'LOC'), (51, 101, 'EMAI...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Jite

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Nikhileshkumar Ikhar
Product development engineer ..." with entities "[(0, 20, 'NAME'), (21, 49, 'DESIG'), (54, 63, 'YOE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Nitin Tr
PeopleSoft Consultant

Bangalore Urban, K..." with entities "[(0, 8, 'NAME'), (32, 47, 'LOC'), (81, 119, 'EMAIL...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Prad

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Santosh Ganta
Senior Systems Engineer - mainframe
..." with entities "[(0, 13, 'NAME'), (14, 37, 'DESIG'), (51, 60, 'LOC...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Sarfaraz Ahmad
Associate network engineer - TATA C..." with entities "[(0, 14, 'NAME'), (15, 41, 'DESIG'), (44, 68, 'COM...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Sent

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "VARUN AHLUWALIA
Quantitative Analyst

- Email me o..." with entities "[(0, 15, 'NAME'), (16, 36, 'DESIG'), (61, 106, 'EM...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Vijayalakshmi Govindarajan
SAP as a Consultant - S..." with entities "[(0, 26, 'NAME'), (27, 47, 'DESIG'), (49, 58, 'COM...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Vika

...

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "B. Gokul
Gokul, Uttar Pradesh - Email me on Indeed..." with entities "[(0, 8, 'LOC'), (9, 14, 'LOC'), (51, 90, 'EMAIL'),...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Krishna Prasad
Patna, Bihar - Email me on Indeed: ..." with entities "[(0, 14, 'NAME'), (15, 20, 'LOC'), (50, 94, 'EMAIL...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Saur

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Jaspreet Kaur
Oceanic Consultants as a HR Executiv..." with entities "[(0, 13, 'NAME'), (14, 33, 'COMPANY'), (38, 51, 'C...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Somanath Behera
Associate, Cognizant technology So..." with entities "[(0, 15, 'NAME'), (16, 25, 'DESIG'), (27, 57, 'COM...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Ashi

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Arpit Godha
Senior Process Executive

Jaipur, Raja..." with entities "[(0, 11, 'NAME'), (12, 37, 'DESIG'), (38, 44, 'LOC...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Jatin Arora
SDET Automation Engineer, Infosys - CR..." with entities "[(0, 11, 'NAME'), (12, 36, 'DESIG'), (37, 46, 'COM...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Kart

<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Moumita Mitra
- Email me on Indeed: indeed.com/r/M..." with entities "[(0, 13, 'NAME'), (35, 80, 'EMAIL'), (1465, 1476, ...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Suman Biswas
SAP UI5 Lead, Native HANA Developer -..." with entities "[(0, 12, 'NAME'), (13, 49, 'DESIG'), (50, 69, 'COM...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tag = biluo_tags_from_offsets(doc, entities)
<ipython-input-12-e145e49fad27>:11: UserWarning: [W030] Some entities could not be aligned in the text "Vala

(781, 781)

In [14]:
tag_vals = set(['X', '[CLS]', '[SEP]'])
for i in range(len(tags)):
    tag_vals = tag_vals.union(tags[i])
tag_vals

{'B-CLG',
 'B-COMPANY',
 'B-DEG',
 'B-DESIG',
 'B-EMAIL',
 'B-GRADYEAR',
 'B-LOC',
 'B-NAME',
 'B-SKILLS',
 'B-YOE',
 'I-CLG',
 'I-COMPANY',
 'I-DEG',
 'I-DESIG',
 'I-EMAIL',
 'I-GRADYEAR',
 'I-LOC',
 'I-NAME',
 'I-SKILLS',
 'I-YOE',
 'L-CLG',
 'L-COMPANY',
 'L-DEG',
 'L-DESIG',
 'L-EMAIL',
 'L-GRADYEAR',
 'L-LOC',
 'L-NAME',
 'L-SKILLS',
 'L-YOE',
 'O',
 'U-CLG',
 'U-COMPANY',
 'U-DEG',
 'U-DESIG',
 'U-EMAIL',
 'U-GRADYEAR',
 'U-LOC',
 'U-SKILLS',
 'U-YOE',
 'X',
 '[CLS]',
 '[SEP]'}

In [15]:
tag2idx = {t: i for i, t in enumerate(tag_vals)}
tag2idx

{'B-YOE': 0,
 'L-DEG': 1,
 'L-LOC': 2,
 'B-NAME': 3,
 'L-CLG': 4,
 'B-EMAIL': 5,
 'U-COMPANY': 6,
 'B-CLG': 7,
 'L-DESIG': 8,
 'I-EMAIL': 9,
 'U-EMAIL': 10,
 'U-LOC': 11,
 'I-CLG': 12,
 'O': 13,
 'B-DESIG': 14,
 '[SEP]': 15,
 'U-DEG': 16,
 'B-SKILLS': 17,
 'I-LOC': 18,
 'I-YOE': 19,
 'I-DESIG': 20,
 'U-GRADYEAR': 21,
 'L-NAME': 22,
 'I-GRADYEAR': 23,
 'B-COMPANY': 24,
 'L-SKILLS': 25,
 'U-CLG': 26,
 'B-GRADYEAR': 27,
 'U-SKILLS': 28,
 'I-SKILLS': 29,
 'B-LOC': 30,
 'I-DEG': 31,
 'I-COMPANY': 32,
 'I-NAME': 33,
 'L-GRADYEAR': 34,
 'L-YOE': 35,
 'L-COMPANY': 36,
 'U-DESIG': 37,
 'L-EMAIL': 38,
 'B-DEG': 39,
 'U-YOE': 40,
 'X': 41,
 '[CLS]': 42}

In [16]:
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}
idx2tag

{0: 'B-YOE',
 1: 'L-DEG',
 2: 'L-LOC',
 3: 'B-NAME',
 4: 'L-CLG',
 5: 'B-EMAIL',
 6: 'U-COMPANY',
 7: 'B-CLG',
 8: 'L-DESIG',
 9: 'I-EMAIL',
 10: 'U-EMAIL',
 11: 'U-LOC',
 12: 'I-CLG',
 13: 'O',
 14: 'B-DESIG',
 15: '[SEP]',
 16: 'U-DEG',
 17: 'B-SKILLS',
 18: 'I-LOC',
 19: 'I-YOE',
 20: 'I-DESIG',
 21: 'U-GRADYEAR',
 22: 'L-NAME',
 23: 'I-GRADYEAR',
 24: 'B-COMPANY',
 25: 'L-SKILLS',
 26: 'U-CLG',
 27: 'B-GRADYEAR',
 28: 'U-SKILLS',
 29: 'I-SKILLS',
 30: 'B-LOC',
 31: 'I-DEG',
 32: 'I-COMPANY',
 33: 'I-NAME',
 34: 'L-GRADYEAR',
 35: 'L-YOE',
 36: 'L-COMPANY',
 37: 'U-DESIG',
 38: 'L-EMAIL',
 39: 'B-DEG',
 40: 'U-YOE',
 41: 'X',
 42: '[CLS]'}

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [19]:
def get_tokenized_train_data(sentences, tags):

    tokenized_texts = []
    word_piece_labels = []

    for word_list, label in zip(sentences, tags):
    
        # Add [CLS] at the front
        temp_lable = ['[CLS]']
        temp_token = ['[CLS]']
    
        for word, lab in zip(word_list, label):
            token_list = tokenizer.tokenize(word.text)
            for m, token in enumerate(token_list):
                temp_token.append(token)
                if m == 0:
                    temp_lable.append(lab)
                else:
                    temp_lable.append('X')  
                
        # Add [SEP] at the end
        temp_lable.append('[SEP]')
        temp_token.append('[SEP]')
    
        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_lable)
    
    return tokenized_texts, word_piece_labels

In [20]:
tokenized_texts, word_piece_labels = get_tokenized_train_data(sentences, tags)

In [21]:
print(tokenized_texts[0])
print(word_piece_labels[0])

['[CLS]', 'A', '##b', '##his', '##he', '##k', 'J', '##ha', 'Application', 'Development', 'Associate', '-', 'A', '##cc', '##ent', '##ure', 'Bengal', '##uru', ',', 'Karnataka', '-', 'Em', '##ail', 'me', 'on', 'Indeed', ':', 'indeed', '.', 'com', '/', 'r', '/', 'A', '##b', '##his', '##he', '##k', '-', 'J', '##ha', '/', '10', '##e', '##7', '##a', '##8', '##c', '##b', '##7', '##32', '##b', '##c', '##43', '##a', '•', 'To', 'work', 'for', 'an', 'organization', 'which', 'provides', 'me', 'the', 'opportunity', 'to', 'improve', 'my', 'skills', 'and', 'knowledge', 'for', 'my', 'individual', 'and', 'company', "'", 's', 'growth', 'in', 'best', 'possible', 'ways', '[SEP]']
['[CLS]', 'B-NAME', 'X', 'X', 'X', 'X', 'L-NAME', 'X', 'B-DESIG', 'I-DESIG', 'L-DESIG', 'O', 'U-COMPANY', 'X', 'X', 'X', 'U-LOC', 'X', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'B-EMAIL', 'I-EMAIL', 'I-EMAIL', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X',

In [22]:
MAX_LEN = 512
bs = 4

In [23]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))
print(input_ids[0])

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (679 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (977 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (567 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1054 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1231 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length

512
[  101   138  1830 27516  4638  1377   147  2328 22491  3273  9666   118
   138 19515  3452  3313  7756 12328   117 12247   118 18653 11922  1143
  1113 10364   131  5750   119  3254   120   187   120   138  1830 27516
  4638  1377   118   147  2328   120  1275  1162  1559  1161  1604  1665
  1830  1559 17101  1830  1665 25631  1161   794  1706  1250  1111  1126
  2369  1134  2790  1143  1103  3767  1106  4607  1139  4196  1105  3044
  1111  1139  2510  1105  1419   112   188  3213  1107  1436  1936  3242
   102     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     

In [24]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels], maxlen=MAX_LEN, value=tag2idx["O"], 
                     padding="post", dtype="long", truncating="post")
print(len(tags[0]))
print(tags[0])

512
[42  3 41 41 41 41 22 41 14 20  8 13  6 41 41 41 11 41 13 13 13 13 41 13
 13  5  9  9 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 41 13 13 13 13 13 15 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 1

In [25]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [26]:
tr_inputs, val_inputs, tr_tags, val_tags, tr_masks, val_masks = train_test_split(input_ids, tags, attention_masks, random_state=2020, 
                                                                                 test_size=0.3)

In [27]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [28]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [29]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx))

In [30]:
# model.cuda()

In [31]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [32]:
epochs = 10
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:   0%|                                                                                    | 0/10 [00:05<?, ?it/s]


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.IntTensor instead (while checking arguments for embedding)

In [ ]:
model.eval()

y_true = []
y_pred = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch

    with torch.no_grad():
        logits = model(input_ids, token_type_ids=None, attention_mask=input_mask,)

    logits = logits.detach().cpu().numpy()
    logits = [list(p) for p in np.argmax(logits, axis=2)]
    
    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()
    
    for i,mask in enumerate(input_mask):
        temp_1 = [] # Real one
        temp_2 = [] # Predict one
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if idx2tag[label_ids[i][j]] != "X" and idx2tag[label_ids[i][j]] != "[CLS]" and idx2tag[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(idx2tag[label_ids[i][j]])
                    temp_2.append(idx2tag[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)
    
print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

print(classification_report(y_true, y_pred,digits=4))